# Analysez vos données textuelles

le cours d'initiation d'OC : https://openclassrooms.com/fr/courses/4470541-analysez-vos-donnees-textuelles 

## Enoncé du premier proejt
Entraînez-vous à prétraiter un corpus en vue de créer un moteur de résumés

Vous allez nettoyer le texte et de créer un jeu de données d’entraînement en vue de créer un moteur de résumé automatique. Voici le dataset que vous allez utiliser : https://cs.nyu.edu/~kcho/DMQA/

Consigne

Vous devrez effectuer les opérations de traitement suivantes sur le texte, pas forcément dans cet ordre
- Créer des paires de documents (article, highlights)
- Suppression de la ponctuation
- Séparation en token en minuscules
- Suppression des stopwords pour les articles
- Calcul des fréquences et tf-idf sur les deux types de documents
- Enregistrement du nouveau jeu de données d’entraînement pour usage ultérieur

## Créer des paires de documents (article, highlights)

### Exemple

Première étape: créer une paire de documents à partir d'un fichier texte. Quand on regarde le contenu on voit que les fichiers sont structurés de manière similaire:

```
(CNN) -- Le contenu de l'article

@highlight

un point important

@highlight

un autre point important
```

A partir de ça comment avoir un contenant python du genre tuple:

`("Le contenu de l'article", "un point important un autre point important")`

In [1]:
contenu_factice = """(CNN) -- Le contenu de l'article

@highlight

un point important

@highlight

un autre point important
"""

In [2]:
contenu_factice.split("@highlight")

["(CNN) -- Le contenu de l'article\n\n",
 '\n\nun point important\n\n',
 '\n\nun autre point important\n']

In [3]:
# '(CNN) --'
contenus = contenu_factice.split("@highlight")
# Dans le cas où on a '(CNN) --', on enlève tout ce qu'il y a devant
article = contenus[0].split("(CNN) --")  # Liste des différents contenus dans article
if len(article)>1:
    article = article[1]  # transformation de la liste en chaîne de caractères
else:
    article = article[0]

article

" Le contenu de l'article\n\n"

In [4]:
# on concatène l'ensemble des autres contenus dans une variable highlights
highlights = ""
for i in range(1, len(contenus)):
    highlights += contenus[i]
highlights

'\n\nun point important\n\n\n\nun autre point important\n'

In [5]:
# Notre structure de données pour un document:
(article, highlights)

(" Le contenu de l'article\n\n",
 '\n\nun point important\n\n\n\nun autre point important\n')

In [6]:
# notre fonction
def conversion_du_document(texte):
    contenus = texte.split("@highlight")
    # Dans le cas où on a '(CNN) --', on enlève tout ce qu'il y a devant
    article = contenus[0].split("(CNN) --")  # Liste des différents contenus dans article
    if len(article)>1:
        article = article[1]  # transformation de la liste en chaîne de caractères
    else:
        article = article[0]
    highlights = ""
    for i in range(1, len(contenus)):
        highlights += contenus[i]
    return (article, highlights)

### Travail sur l'échantillon des données

[Listing All Files in a Directory](https://realpython.com/working-with-files-in-python/#listing-all-files-in-a-directory)

In [7]:
import os

# List all files in a directory using os.listdir
basepath = './cnn/stories/'
for entry in os.listdir(basepath):
    if os.path.isfile(os.path.join(basepath, entry)) and entry.endswith('.story'):
        print(entry)

0a0f56ebc5a0a67ed18de79d99b40a42d8058d04.story
0a1ad82d161d90d758240407cb8c8fcebff4a212.story
0a0b44620d0dec6fdfc1aa139ff51bcb36c56c51.story
0a0a4c90d59df9e36ffec4ba306b4f20f3ba4acb.story
0a001b54b5b0ea17f11aa8d7acafb40ae7748ca6.story
0a0adc84ccbf9414613e145a3795dccc4828ddd4.story
0a000f521734500aa360bf6bb2bb31446e4d66cd.story
0a0d11c125e4e4b061061f44b08f60cb5b8bf177.story
0a0b59738a88e97ef66322a8c866d22ebf079289.story
0a0aa464d262b903f44b0f8eaa67f13dd1946cfd.story
0a0c2f4e07ba05f3226e8afec1350ac82161cd2e.story


In [8]:
# Création de l'ensemble des paires de documents:
paires = []
basepath = './cnn/stories/'
for entry in os.listdir(basepath):
    if os.path.isfile(os.path.join(basepath, entry)) and entry.endswith('.story'):
        with open(os.path.join(basepath, entry)) as f:
            texte = f.read()
            #print(texte)
            paires.append(conversion_du_document(texte))
paires[:2]

[(' The man suspected of trying to attack the Dutch royal family by crashing his car near their bus has died, Dutch police said Friday.\n\nA car is pictured after crashing into the crowd waiting for the visit of the royal family in Apeldoorn.\n\nThe man, whose name was not released, had been seriously injured in the crash Thursday in the town of Apeldoorn, about 45 miles east of Amsterdam, police said. He died early Friday, police said.\n\nHe had been charged with trying to attack the royal family, authorities said. Were you there? Send us your video, images\n\nPolice searched the man\'s house but found no weapons, explosives, "or any other clues that could lead to the involvement of other people," police said.\n\nFive people, as well as the driver, were killed and 12 were wounded in the incident, which happened during the country\'s annual Queen\'s Day celebration, police said.\n\nCrowds had lined the streets to see Queen Beatrix and her family ride by in an open-top bus during the Ne

## Traitement du texte
- Suppression de la ponctuation
- Séparation en token en minuscules
- Suppression des stopwords pour les articles

In [9]:
text ='The U.N. Security Council approved a resolution Monday to send 4,200 peacekeepers to Abyei, Sudan, as part of a recent agreement between Sudan and Southern Sudan.\n\nThe resolution will establish, for six months, the United Nations Interim Security Force for Abyei (UNISFA), comprising "a maximum of 4,200 military personnel, 50 police personnel, and appropriate civilian support," the resolution states.\n\n'

In [10]:
#Suppression de la ponctuation
import string
''.join([i for i in text if i not in string.punctuation])

'The UN Security Council approved a resolution Monday to send 4200 peacekeepers to Abyei Sudan as part of a recent agreement between Sudan and Southern Sudan\n\nThe resolution will establish for six months the United Nations Interim Security Force for Abyei UNISFA comprising a maximum of 4200 military personnel 50 police personnel and appropriate civilian support the resolution states\n\n'

In [11]:
import nltk
nltk.download('punkt')
text_without_punctuation = ''.join([i for i in text if i not in string.punctuation])

nltk.word_tokenize(text_without_punctuation.lower())

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pierrejaumier/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['the',
 'un',
 'security',
 'council',
 'approved',
 'a',
 'resolution',
 'monday',
 'to',
 'send',
 '4200',
 'peacekeepers',
 'to',
 'abyei',
 'sudan',
 'as',
 'part',
 'of',
 'a',
 'recent',
 'agreement',
 'between',
 'sudan',
 'and',
 'southern',
 'sudan',
 'the',
 'resolution',
 'will',
 'establish',
 'for',
 'six',
 'months',
 'the',
 'united',
 'nations',
 'interim',
 'security',
 'force',
 'for',
 'abyei',
 'unisfa',
 'comprising',
 'a',
 'maximum',
 'of',
 '4200',
 'military',
 'personnel',
 '50',
 'police',
 'personnel',
 'and',
 'appropriate',
 'civilian',
 'support',
 'the',
 'resolution',
 'states']

> Les chiffres seront peu informatifs, on peut les supprimer aussi

In [12]:
test = '3 ou 4 trucs'
''.join(i for i in test if not i.isdigit())

' ou  trucs'

Suppression des stopwords

In [13]:
nltk.download('stopwords')
from nltk.corpus import stopwords

word_list = nltk.word_tokenize(''.join([i for i in text.lower() if i not in string.punctuation and not i.isdigit()]))

[word for word in word_list if word not in stopwords.words('english')]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pierrejaumier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['un',
 'security',
 'council',
 'approved',
 'resolution',
 'monday',
 'send',
 'peacekeepers',
 'abyei',
 'sudan',
 'part',
 'recent',
 'agreement',
 'sudan',
 'southern',
 'sudan',
 'resolution',
 'establish',
 'six',
 'months',
 'united',
 'nations',
 'interim',
 'security',
 'force',
 'abyei',
 'unisfa',
 'comprising',
 'maximum',
 'military',
 'personnel',
 'police',
 'personnel',
 'appropriate',
 'civilian',
 'support',
 'resolution',
 'states']

## Sac de mots
Association mot - fréquence dans un texte

Counter : https://docs.python.org/3/library/collections.html#collections.Counter

In [14]:
from collections import Counter
Counter(['crêpe', 'brioche', 'brioche'])

Counter({'crêpe': 1, 'brioche': 2})

In [15]:
filtered_words = [word for word in word_list if word not in stopwords.words('english')]
bag_of_words = Counter(filtered_words)
bag_of_words

Counter({'un': 1,
         'security': 2,
         'council': 1,
         'approved': 1,
         'resolution': 3,
         'monday': 1,
         'send': 1,
         'peacekeepers': 1,
         'abyei': 2,
         'sudan': 3,
         'part': 1,
         'recent': 1,
         'agreement': 1,
         'southern': 1,
         'establish': 1,
         'six': 1,
         'months': 1,
         'united': 1,
         'nations': 1,
         'interim': 1,
         'force': 1,
         'unisfa': 1,
         'comprising': 1,
         'maximum': 1,
         'military': 1,
         'personnel': 2,
         'police': 1,
         'appropriate': 1,
         'civilian': 1,
         'support': 1,
         'states': 1})

In [16]:
article = paires[0][0]
word_list = nltk.word_tokenize(''.join([i for i in article.lower() if i not in string.punctuation and not i.isdigit()]))

filtered_words = [word for word in word_list if word not in stopwords.words('english')]
bag_of_words = Counter(filtered_words)
bag_of_words

Counter({'man': 2,
         'suspected': 1,
         'trying': 2,
         'attack': 4,
         'dutch': 4,
         'royal': 9,
         'family': 10,
         'crashing': 2,
         'car': 5,
         'near': 1,
         'bus': 4,
         'died': 2,
         'police': 7,
         'said': 8,
         'friday': 3,
         'pictured': 1,
         'crowd': 1,
         'waiting': 1,
         'visit': 1,
         'apeldoorn': 2,
         'whose': 1,
         'name': 1,
         'released': 1,
         'seriously': 1,
         'injured': 1,
         'crash': 3,
         'thursday': 1,
         'town': 1,
         'miles': 1,
         'east': 1,
         'amsterdam': 2,
         'early': 1,
         'charged': 1,
         'authorities': 1,
         'send': 1,
         'us': 1,
         'video': 1,
         'images': 1,
         'searched': 1,
         'mans': 1,
         'house': 2,
         'found': 1,
         'weapons': 1,
         'explosives': 1,
         'clues': 1,
         'could

## Création du nouveau jeu de données

In [17]:
def create_bag_of_words(text):
    word_list = nltk.word_tokenize(''.join([i for i in text.lower() if i not in string.punctuation and not i.isdigit()]))
    filtered_words = [word for word in word_list if word not in stopwords.words('english')]
    bag_of_words = Counter(filtered_words)
    return bag_of_words

In [18]:
pairs_as_BOW = [ (create_bag_of_words(article), create_bag_of_words(highlights)) \
                  for article, highlights in paires ]
pairs_as_BOW[0]

(Counter({'man': 2,
          'suspected': 1,
          'trying': 2,
          'attack': 4,
          'dutch': 4,
          'royal': 9,
          'family': 10,
          'crashing': 2,
          'car': 5,
          'near': 1,
          'bus': 4,
          'died': 2,
          'police': 7,
          'said': 8,
          'friday': 3,
          'pictured': 1,
          'crowd': 1,
          'waiting': 1,
          'visit': 1,
          'apeldoorn': 2,
          'whose': 1,
          'name': 1,
          'released': 1,
          'seriously': 1,
          'injured': 1,
          'crash': 3,
          'thursday': 1,
          'town': 1,
          'miles': 1,
          'east': 1,
          'amsterdam': 2,
          'early': 1,
          'charged': 1,
          'authorities': 1,
          'send': 1,
          'us': 1,
          'video': 1,
          'images': 1,
          'searched': 1,
          'mans': 1,
          'house': 2,
          'found': 1,
          'weapons': 1,
          'explosiv

In [19]:
Counter('abracadabra')

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [20]:
Counter('abracadabra').most_common(2)

[('a', 5), ('b', 2)]